In [ ]:
# install required packages

! pip install -qU langchain-community pymupdf
! pip install -qU langchain_huggingface
! pip install -qU sentence-transformers
! pip install -qU langchain-perplexity
! pip install -qU faiss-cpu
! pip install gradio
! pip install kaggle

In [1]:
# import libraries

import os, glob
from pathlib import Path
from langchain_community.document_loaders import PyMuPDFLoader, CSVLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from sentence_transformers import CrossEncoder, util
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_perplexity import ChatPerplexity
from langchain.retrievers import ContextualCompressionRetriever
from langchain.memory import ConversationBufferMemory
from langchain.retrievers.document_compressors import CrossEncoderReranker
from langchain_community.cross_encoders import HuggingFaceCrossEncoder
from langchain.chains import LLMChain
from langchain_core.messages import HumanMessage, AIMessage
import gradio as gr

/Users/RajivGaba/aiml_projects/Semantic Spotter/.venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Set the key for LLM API

try:
    from google.colab import userdata
    os.environ['PERPLEXITY_API_KEY'] = userdata.get('PPLX_API_KEY_2')
except:
    import dotenv
    dotenv.load_dotenv()
    os.environ['PERPLEXITY_API_KEY'] = os.getenv('PPLX_API_KEY')

In [3]:
config = {
    'data_path' : '/Users/RajivGaba/aiml_projects/Semantic Spotter/Data/',
    'chunk_size' : 512,
    'chunk_overlap' : 80,
    'vector_store_name' : "faiss_myntra_db",
    'embedding_model' : 'all-MiniLM-L6-v2',
    'refresh_vector_store' : 'N',
    'PPLX_API_KEY' : os.getenv('PERPLEXITY_API_KEY'),
    'domain' : 'fashion',
    'chat_model' : "sonar-pro",
    'rerank_model' : 'BAAI/bge-reranker-base'
}

In [ ]:
# Download dataset from kaggle using kaggle API
from kaggle.api.kaggle_api_extended import KaggleApi

api = KaggleApi()
api.authenticate()
# api.dataset_download_files('promptcloud/myntra-e-commerce-product-data-november-2023', path='./data/', unzip=True)
# api.dataset_download_files("ronakbokaria/myntra-products-dataset", path='./data/', unzip=True)
api.dataset_download_files("djagatiya/myntra-fashion-product-dataset", path='./data/', unzip=True)

In [5]:
# Define reusable functions

def get_data_chunks(folder_path):
  """
  This function will create chunks from the files present in the dataset.
  This takes directory where the files exists. Basis the type of file i.e.
  PDF, CSV or text, a loader is initialised and chunks are created from
  content of the data
  """
  # loader = PyMuPDFLoader(pdf_file)
  # documents = loader.load()

  all_documents = []

  # Loop through all files in the folder
  for file_path in folder_path.iterdir():
      if file_path.suffix.lower() == ".pdf":
          loader = PyMuPDFLoader(str(file_path))
      elif file_path.suffix.lower() == ".csv":
          loader = CSVLoader(str(file_path))
      elif file_path.suffix.lower() == ".txt":
          loader = TextLoader(str(file_path))
      else:
          continue  # Skip unsupported file types

      # Load and append documents
      documents = loader.load()
      all_documents.extend(documents)

  # chunking/splitting
  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=config['chunk_size'],
      chunk_overlap=config['chunk_overlap'],
      strip_whitespace=True,
      separators=["\n\n", "\n", " ", ""]
  )
  text_chunks = text_splitter.split_documents(documents=all_documents)
  return text_chunks

In [6]:
def get_embeddings_model():
  embedding_model = HuggingFaceEmbeddings(
      model_name=config['embedding_model'],
      show_progress=True,
      multi_process=True,
      model_kwargs={'device': 'mps'}
  )
  return embedding_model

In [7]:
def create_vector_store(text_chunks, embedding_model):
  if config['refresh_vector_store'] == 'Y' or not os.path.exists(config['vector_store_name']):
      vector_store = FAISS.from_documents(text_chunks, embedding_model)
      vector_store.save_local(config['vector_store_name'])
  else:
      vector_store = FAISS.load_local(config['vector_store_name'], embedding_model, allow_dangerous_deserialization=True)
  return vector_store

In [8]:
def create_chat_client():
  return ChatPerplexity(
      temperature=0,
      pplx_api_key=config['PPLX_API_KEY'], # Pass the API key explicitly
      model=config['chat_model']
  )

In [9]:
def get_retriever(top_k=5):
  retriever = vector_store.as_retriever(search_kwargs={'k': top_k})
  return retriever

In [10]:
def get_reranked_query_results(query):
  model = HuggingFaceCrossEncoder(model_name=config['rerank_model'])

  compressor = CrossEncoderReranker(model=model, top_n=5)

  compression_retriever = ContextualCompressionRetriever(
      base_compressor=compressor,
      base_retriever=get_retriever()
  )

  compressed_docs = compression_retriever.invoke(query)

  return compressed_docs

In [ ]:
def rag_pipeline(user_question, history):

  print(f"{history}")

  # Retrieve relevant documents from the vector store based on the user's question.
  retrieved_documents = get_reranked_query_results(user_question)

  # Extract the page content from the retrieved documents and join them into a single context string.
  formatted_context = "\n\n".join(doc.page_content for doc in retrieved_documents)

  # Call the large language model with the user's question and the formatted context.
  answer = get_llm_response(user_question, formatted_context, config['domain'])
  return answer

In [26]:
def rag_with_history_pipeline(new_user_message, gradio_history, llm , retriever, system_message):
    # --- STEP 1: Load Gradio history into LangChain Memory ---
    # Gradio sends history in the format: [[user_msg, bot_msg], [user_msg, bot_msg], ...]
    # We must reset the memory and rebuild it for each new call,
    # as Gradio manages the state outside of LangChain's memory object.

    # Clear memory from previous (potentially different user) sessions
    memory.clear()

    # Convert Gradio format to LangChain Messages and load into memory
    for message_pair in gradio_history:
        # message_pair[0] is User message, message_pair[1] is Assistant message
        user_text = message_pair[0]
        ai_text = message_pair[1]

        # Add the conversation turn to the memory object
        memory.chat_memory.add_user_message(user_text)
        memory.chat_memory.add_ai_message(ai_text)


    # --- STEP 2: RAG Logic (Retrieve context using the new question AND the history) ---
    # Your current retrieval logic needs the new question (new_user_message)
    # AND the context from the conversation history to perform better retrieval.

    # Create the full query string by joining the last few messages
    # This helps the retriever find context relevant to the full conversation.
    if gradio_history:
        last_turn = f"User: {gradio_history[-1][0]} | Assistant: {gradio_history[-1][1]}"
        full_query = f"{last_turn} | New Question: {new_user_message}"
    else:
        full_query = new_user_message

    # Run your vector store retrieval using 'full_query'
    final_retrieved_docs = retriever.invoke(full_query)

    # Format the retrieved documents into the context string
    formatted_context = "\n".join([doc.page_content for doc in final_retrieved_docs])

    # NOTE: You'll need to update your final prompt template to include {context}
    # and pass the formatted_context into the invoke call.

    prompt = ChatPromptTemplate.from_messages(
        [
            ("system",{system_message},), MessagesPlaceholder(variable_name="chat_history"), ("human", "{full_query}"),
        ]
    )

    # --- STEP 3: Invoke the Chain ---
    # The chain automatically pulls 'chat_history' from memory,
    # and inserts 'input' and 'context' (if you update the prompt)
    
    chain = prompt | llm
    response = chain.invoke(

    )

    # The final memory state is now updated internally by the chain.
    # The Gradio interface will update its displayed history based on the returned string.
    return response['text']

In [27]:
query  = "i am looking for an outfit for hill station vacation"
# Initialize LLM chat model

llm = create_chat_client()

# 1. Define the Memory Component
# The memory stores the history (user/assistant messages)
# It uses 'chat_history' as the key to insert into the prompt template

memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# 2. Define the Prompt Template
# Use MessagesPlaceholder to tell LangChain where to inject the list of messages

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful fashion assistant. Use the following context to answer the user's question. If you don't know the answer, say so. Keep the conversation flowing.",
        ),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{query}"),
    ]
)

# 3. Define a simple Chain (or your RAG chain wrapper)
# For this simplified example, we'll use a basic LLMChain
chain = prompt | llm
response = chain.invoke(
    {
        "query": query,
        "chat_history": []
    }
)


In [23]:
print(response.content)

For a hill station vacation, the best outfit combines **layered clothing, comfortable bottoms, and sturdy footwear** to adapt to changing temperatures and terrain[1][2]. Start with a **breathable base layer** like a cotton shirt or thermal top, add a **sweater or cardigan** for warmth, and finish with a **jacket or coat** for chilly mornings and evenings[1][2].

**Key outfit elements:**
- **Trousers, hiking pants, or joggers** for comfort and mobility[1].
- **Sweaters, hoodies, or light wool pullovers** for warmth[1][2].
- **Denim jeans or shorts** paired with flannel shirts or blouses for a casual look[2].
- **Dresses or maxi dresses** with cardigans or jackets for a versatile, stylish option (especially in summer)[1][3].
- **Sturdy boots or hiking shoes** for walking and exploring; sneakers or espadrilles for relaxed outings[1][2].
- **Accessories** like a wide-brimmed hat, scarf, and a functional backpack for sun protection and convenience[2].

**Color and style tips:**
- **Bright c

In [ ]:

chunked_data = []
folder_path = Path(config['data_path'])
# folder_path = Path("/content/data/")

# Step 1: Generate chunks from the dataset
if config['refresh_vector_store'] == 'Y':
    chunked_data = get_data_chunks(folder_path)

# Step 2: Create embeddings and put them into a vector store
embedding_model = get_embeddings_model()
vector_store = create_vector_store(chunked_data, embedding_model)

In [ ]:
get_reranked_query_results("i need a new outfit")
llm = create_chat_client()

In [28]:
iface = gr.ChatInterface(
    fn=rag_with_history_pipeline,
    type = 'messages',
    chatbot=gr.Chatbot(height=500),
    textbox=gr.Textbox(placeholder="Help me find a formal shirt..", container=False, scale=7),
    title="ClosetAI - Fashion Studio",
    description="Ask ClosetAI anything about fashion products on Mytra",
    theme="glass",
    # examples=["Blazers for men", "Party dresses for women", "Athleisure" , "Sports attire" ],
    cache_examples=True,
)
iface.launch(share=True, debug=True)

/var/folders/w2/6035x_495mg87nyn6pl05mp40000gq/T/ipykernel_11993/2186140973.py:4: UserWarning: You have not specified a value for the `type` parameter. Defaulting to the 'tuples' format for chatbot messages, but this is deprecated and will be removed in a future version of Gradio. Please set type='messages' instead, which uses openai-style dictionaries with 'role' and 'content' keys.
  chatbot=gr.Chatbot(height=500),
/Users/RajivGaba/aiml_projects/Semantic Spotter/.venv/lib/python3.13/site-packages/gradio/chat_interface.py:323: UserWarning: The type of the gr.Chatbot does not match the type of the gr.ChatInterface.The type of the gr.ChatInterface, 'messages', will be used.
  warnings.warn(
/Users/RajivGaba/aiml_projects/Semantic Spotter/.venv/lib/python3.13/site-packages/gradio/utils.py:1052: UserWarning: Expected 5 arguments for function <function rag_with_history_pipeline at 0x1788fe7a0>, received 2.
  warnings.warn(
/Users/RajivGaba/aiml_projects/Semantic Spotter/.venv/lib/python3.1

* Running on local URL:  http://127.0.0.1:7860


KeyboardInterrupt: 

In [29]:
iface.close()

Closing server running on port: 7860
